# Model VGG16

- Prosty model wyrózniajacy sie prostą i elegancką archtekturą zbudowaną z kolejnych warstw konwolucyjnych
- Model zawiera 16 warstw uczących sie wag (13 konwolucyjnych + 3 w pełni połączone warstwy
- Wszystkie wartwy konwolucyjne używają filtrów o wymiarach 3x3

In [1]:
import pickle
from tensorflow.keras.utils import to_categorical
import pandas as pd
from keras.callbacks import Callback, EarlyStopping ,ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.preprocessing import image
from livelossplot import PlotLossesKeras 
import cv2
import numpy as np
np.random.seed(0)




### Wczytywanie danych

In [2]:
train_dir = r"..\input\train"
test_dir = r"..\input\test"

### Tworzenie generatora danych

In [3]:
datagen = ImageDataGenerator(rescale = 1.0/255.0, validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Number of classes: 7


In [4]:
print(f"Train class indices: {train_generator.class_indices}")
print(f"Validation class indices: {validation_generator.class_indices}")

Train class indices: {'angry': 0, 'disgusted': 1, 'fearful': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprised': 6}
Validation class indices: {'angry': 0, 'disgusted': 1, 'fearful': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprised': 6}


### Ładowanie modelu i konfiguracja modelu bazowego
 usuwamy warstwy klsyfikacyjne

In [5]:
base_model = VGG16(weights='imagenet', include_top=False,input_shape = (128,128,3) )

In [6]:
#Zamrozenie wszystkich warstw
for layer in base_model.layers:
    layer.trainable = False

i = 15

# Odmrazanie ostatnich i warstw
for layer in base_model.layers[-i:]:
    layer.trainable = True

#Sprawdzenie statusu warstw
for i, layer in enumerate(base_model.layers):
    print(f' Layer {i} : {layer.name}, Trainable: {layer.trainable}')





 Layer 0 : input_layer, Trainable: False
 Layer 1 : block1_conv1, Trainable: False
 Layer 2 : block1_conv2, Trainable: False
 Layer 3 : block1_pool, Trainable: False
 Layer 4 : block2_conv1, Trainable: True
 Layer 5 : block2_conv2, Trainable: True
 Layer 6 : block2_pool, Trainable: True
 Layer 7 : block3_conv1, Trainable: True
 Layer 8 : block3_conv2, Trainable: True
 Layer 9 : block3_conv3, Trainable: True
 Layer 10 : block3_pool, Trainable: True
 Layer 11 : block4_conv1, Trainable: True
 Layer 12 : block4_conv2, Trainable: True
 Layer 13 : block4_conv3, Trainable: True
 Layer 14 : block4_pool, Trainable: True
 Layer 15 : block5_conv1, Trainable: True
 Layer 16 : block5_conv2, Trainable: True
 Layer 17 : block5_conv3, Trainable: True
 Layer 18 : block5_pool, Trainable: True


## Budowa modelu

In [7]:

model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

optimizer = Adam(0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### Konfiguracja szkolenia

In [8]:
callbacks = [
    PlotLossesKeras(),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    
    ModelCheckpoint(filepath= r'..\models\emotion_model_VGG16_model_{epoch:02d}_{val_loss:.2f}.keras', 
                    monitor='val_loss', save_best_only=True, mode='min' , verbose=1),
    ReduceLROnPlateau( monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_lr=1e-6 ) 
]



In [ ]:
history = model.fit(
        train_generator,
        epochs=50,
        callbacks=callbacks,
        validation_data=validation_generator
    )

Epoch 1/50


C:\programy\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 88/718 ━━━━━━━━━━━━━━━━━━━━ 2:00:09 11s/step - accuracy: 0.2098 - loss: 4.5616

## Ocena modelu na danych testowych

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator, verbose=1)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

## Wnioski

Po zwiekszeniu obrazu z 64x64x3 na 128x128x3